In [1]:
import keras
from keras.models import load_model
deepsea_model = load_model("/srv/scratch/avanti/ExPecto/resources/deepseabeluga_keras.h5")

Using TensorFlow backend.
W0822 16:31:40.468896 140525122139904 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0822 16:31:40.496711 140525122139904 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0822 16:31:40.567846 140525122139904 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0822 16:31:40.568900 140525122139904 deprecation_wrapper.py:119] From /users/avanti/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Pl

In [12]:
import seqdataloader
from seqdataloader.batchproducers.coordbased.coordbatchproducers import SimpleCoordsBatchProducer
from seqdataloader.batchproducers.coordbased.coordstovals.fasta import PyfaidxCoordsToVals


positives_coords_batch_producer = SimpleCoordsBatchProducer(
    bed_file="2000bp_around_hg38_new_k562.narrowPeak.gz",
    batch_size=64)
negatives_coords_batch_producer = SimpleCoordsBatchProducer(
    bed_file="hg38_merged_universal_neg_representative_peaks.bed.gz",
    batch_size=64)
coords_to_onehot = PyfaidxCoordsToVals(genome_fasta_path="hg38.fasta")



Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer
Heads up: coordinates in bed file are assumed to be on the positive strand; if strand in the bed file is improtant to you, please add that feature to SimpleCoordsBatchProducer


In [ ]:
positives_preds = []
for batch_idx in range(len(positives_coords_batch_producer)):
    if (batch_idx%100==0):
        print("Done",len(positives_preds))
    positives_preds.extend(deepsea_model.predict(
                            coords_to_onehot(positives_coords_batch_producer[batch_idx])))

Done 0
Done 6400
Done 12800
Done 19200
Done 25600
Done 32000
Done 38400
Done 44800
Done 51200
Done 57600
Done 64000
Done 70400
Done 76800
Done 83200
Done 89600
Done 96000
Done 102400
Done 108800
Done 115200
Done 121600
Done 128000


In [ ]:
negatives_preds = []
for batch_idx in range(len(negatives_coords_batch_producer)):
    if (batch_idx%100==0):
        print("Done",len(negatives_preds))
    negatives_preds.extend(deepsea_model.predict(
                            coords_to_onehot(negatives_coords_batch_producer[batch_idx])))

In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np
y_score = positives_preds+negatives_preds
y_true = np.array([1 for x in positives_preds]
                  +[0 for x in negatives_preds])

In [8]:
deepsea_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1993, 320)         10560     
_________________________________________________________________
activation_1 (Activation)    (None, 1993, 320)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1986, 320)         819520    
_________________________________________________________________
activation_2 (Activation)    (None, 1986, 320)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1986, 320)         0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 496, 320)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 489, 480)          1229280   
__________